In [1]:
import robotic as ry
import numpy as np
from visual import getObject, lookAtObj, scanObject, piece_pci
from arena import *
import open3d as o3d
import random

INITIAL_OBJ_POS = [-.5, 0, .69]
ON_REAL = False
ARENA_POS = np.array([-.54, -.17, .651])

GRIPPER_FINGER_SEPARATION = .075
GRIPPER_FINGER_WIDTH = .025

def validSideDistance(p1, p2, n):
    dist = np.linalg.norm(p2-p1)
    normal_dist = abs(np.dot(p1 - p2, n) / np.linalg.norm(n))
    side_dist = np.sqrt(dist*dist-normal_dist*normal_dist)
    return side_dist <= GRIPPER_FINGER_WIDTH*.5

def validNormalDistance(p1, p2, n, dist_thresh=GRIPPER_FINGER_SEPARATION):
    return (abs(np.dot(p1 - p2, n) / np.linalg.norm(n)) <= dist_thresh
            and abs(np.dot(p1 - p2, n) / np.linalg.norm(n)) >= GRIPPER_FINGER_WIDTH)

def validNormalRelativeDir(p1, p2, n):
    p2pvec = p2-p1
    dot_product = np.dot(n, p2pvec)
    # Normal vector has magnitude one so we ignore it
    magnitude_vector = np.linalg.norm(p2pvec)
    cosine_angle = dot_product / magnitude_vector
    return cosine_angle <= 0

def validNormalDir(n1, n2):
    upvec = np.array([0., 0., 1.])
    dot_product1 = np.dot(n1, upvec)
    dot_product2 = np.dot(n2, upvec)
    return dot_product1 >= 0 and dot_product2 >= 0



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#-- load parameters, typically automatically from 'rai.cfg'
ry.params_print()
verbose = 0

#-- define a configuration
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

obj = C.addFrame('obj')
obj.setShape(ry.ST.ssBox, size=[.05, .09, .1, .005])
obj.setPosition(INITIAL_OBJ_POS)
obj.setColor([1, .5, .5])
obj.setMass(.1)
obj.setContact(True)

C.addFrame('predicted_obj') \
    .setShape(ry.ST.marker, size=[.1]) \
    .setPosition(INITIAL_OBJ_POS) \
    .setColor([1, 0, 0])

for i in range(32):
    C.addFrame(f'view_point_{i}') \
        .setPosition([0, 0, 0]) \
        .setShape(ry.ST.sphere, size=[.02]) \
        .setColor([1, 1, 0])

bot = ry.BotOp(C, ON_REAL)

obj_pos = INITIAL_OBJ_POS

# Generate Arena
arena = CircularArena(middleP=ARENA_POS, innerR=None, outerR=.3)
arena.plotArena(C)

-- module.cpp:operator():95(0) python,
RealSense/resolution: 640,
RealSense/alignToDepth!,
physx/motorKp: 10000,
physx/motorKd: 1000,
physx/multiBody!


In [3]:
# Point towards set initial object position
lookAtObj(bot, C, np.array(INITIAL_OBJ_POS))

# Capture midpoint from point cloud
obj_pos  = getObject(bot, C, arena) 

scanned_views_ = scanObject(bot, C, np.array(obj_pos), arena, view_count=8)
scanned_views = []
for sv in scanned_views_:
    point_cloud_obj = o3d.geometry.PointCloud()
    point_cloud_obj.points = o3d.utility.Vector3dVector(sv)
    scanned_views.append(point_cloud_obj)

{ time: 0.006117, evals: 10, done: 1, feasible: 1, sos: 0.010296, f: 0, ineq: 0, eq: 0.0113002 }
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'pcl'
{ time: 0.004728, evals: 19, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00304091 }
{ time: 0.002891, evals: 13, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00374233 }
{ time: 0.005945, evals: 12, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.0042577 }
{ time: 0.029588, evals: 59, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.071881 }
{ time: 0.002945, evals: 11, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.0036227 }
{ time: 0.00383, evals: 20, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00308376 }
{ time: 0.092977, evals: 351, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 0.826737 }
{ time: 0.011117, evals: 52, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 0.824008 }


In [4]:
final_points = piece_pci(scanned_views)

Starting loop...
Ending loop...
Starting loop...
Ending loop...
Starting loop...
Ending loop...


In [5]:
pclFrame = C.getFrame("pcl")
pclFrame.setPointCloud(np.asarray(final_points.points))
C.view_recopyMeshes()

In [6]:
def findPossibleGrasps(point_cloud, voxel_size=.005, verbose=3):

    # Prepare point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    if voxel_size: pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    if verbose > 2:
        o3d.visualization.draw_geometries([pcd], point_show_normal=True)
    
    # Find antipodal pairs
    antipodal_pairs = []
    for i in range(len(pcd.points)):
        normal_i = np.asarray(pcd.normals[i])
        for j in range(i + 1, len(pcd.points)):
            normal_j = np.asarray(pcd.normals[j])
            dot_product = np.dot(normal_i, normal_j)
            if (dot_product < -0.7 and
                validSideDistance(pcd.points[i], pcd.points[j], pcd.normals[i]) and
                validNormalDistance(pcd.points[i], pcd.points[j], pcd.normals[i]) and
                validNormalRelativeDir(pcd.points[i], pcd.points[j], pcd.normals[i])
                and validNormalDir(pcd.points[j], pcd.normals[i])
                ):
                    antipodal_pairs.append((i, j))
        if verbose > 2 and (i+1)%100 == 0:
            print(f"Matching atimodal points: {i+1}/{len(pcd.points)} ({((i+1)*100/len(pcd.points)):.2f}%) Done")

    if verbose:
        print("Antipodal pairs found:", antipodal_pairs)

        if verbose > 1:
            # Create a line set to visualize antipodal pairs
            line_set = o3d.geometry.LineSet()

            # Populate line_set with antipodal pairs
            lines = []
            for pair in antipodal_pairs:
                lines.append([pair[0], pair[1]])
            lines = np.array(lines).astype(np.int32)
            line_set.points = o3d.utility.Vector3dVector(np.asarray(pcd.points))
            line_set.lines = o3d.utility.Vector2iVector(lines)

            # Set color of the lines to red
            line_colors = np.array([[1, 0, 0] for _ in range(len(lines))])
            line_set.colors = o3d.utility.Vector3dVector(line_colors)

            # Visualize the point cloud and antipodal pairs
            o3d.visualization.draw_geometries([pcd, line_set], point_show_normal=True)

    return antipodal_pairs

point_indices = findPossibleGrasps(np.asarray(final_points.points), voxel_size=.005, verbose=3)

Matching atimodal points: 100/2024 (4.94%) Done
Matching atimodal points: 200/2024 (9.88%) Done
Matching atimodal points: 300/2024 (14.82%) Done
Matching atimodal points: 400/2024 (19.76%) Done
Matching atimodal points: 500/2024 (24.70%) Done
Matching atimodal points: 600/2024 (29.64%) Done
Matching atimodal points: 700/2024 (34.58%) Done
Matching atimodal points: 800/2024 (39.53%) Done
Matching atimodal points: 900/2024 (44.47%) Done
Matching atimodal points: 1000/2024 (49.41%) Done
Matching atimodal points: 1100/2024 (54.35%) Done
Matching atimodal points: 1200/2024 (59.29%) Done
Matching atimodal points: 1300/2024 (64.23%) Done
Matching atimodal points: 1400/2024 (69.17%) Done
Matching atimodal points: 1500/2024 (74.11%) Done
Matching atimodal points: 1600/2024 (79.05%) Done
Matching atimodal points: 1700/2024 (83.99%) Done
Matching atimodal points: 1800/2024 (88.93%) Done
Matching atimodal points: 1900/2024 (93.87%) Done
Matching atimodal points: 2000/2024 (98.81%) Done
Antipodal p

In [7]:
random_grasp_indices = random.choice(point_indices)
gripping_pos = (final_points.points[random_grasp_indices[0]] + final_points.points[random_grasp_indices[1]]) * .5
gripping_angle = final_points.points[random_grasp_indices[0]] + final_points.points[random_grasp_indices[1]]
gripping_angle /= np.linalg.norm(gripping_angle)

In [8]:
bot.home(C)
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)
    
bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.sync(C)

komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 1, 1., 0)

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

komo.addObjective([1.], ry.FS.position, ['l_gripper'], ry.OT.eq, [1e1], gripping_pos)
komo.addObjective([1.], ry.FS.vectorX, ["l_gripper"], ry.OT.eq, [1e1], gripping_angle)

ret = ry.NLP_Solver().setProblem(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=0).solve()
print(ret)
bot.move(komo.getPath(), [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperClose(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.home(C)
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

{ time: 0.008036, evals: 18, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.0342324 }


In [9]:
bot.home(C)
C.view(True)

0

In [10]:
del bot
del C

-- bot.cpp:~BotOp:122(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
